In [1]:
import hashlib
import os
import pickle
from urllib.request import urlretrieve

import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import resample
from tqdm import tqdm
from zipfile import ZipFile

print('All modules imported.')

def download(url, file):
    """
    Download file from <url>
    :param url: URL to file
    :param file: Local file path
    """
    if not os.path.isfile(file):
        print('Downloading ' + file + '...')
        urlretrieve(url, file)
        print('Download Finished')

# Download the training and test dataset.
download('https://s3.amazonaws.com/udacity-sdc/notMNIST_train.zip', 'notMNIST_train.zip')
download('https://s3.amazonaws.com/udacity-sdc/notMNIST_test.zip', 'notMNIST_test.zip')

# Make sure the files aren't corrupted
assert hashlib.md5(open('notMNIST_train.zip', 'rb').read()).hexdigest() == 'c8673b3f28f489e9cdf3a3d74e2ac8fa',\
        'notMNIST_train.zip file is corrupted.  Remove the file and try again.'
assert hashlib.md5(open('notMNIST_test.zip', 'rb').read()).hexdigest() == '5d3c7e653e63471c88df796156a9dfa9',\
        'notMNIST_test.zip file is corrupted.  Remove the file and try again.'

# Wait until you see that all files have been downloaded.
print('All files downloaded.')

def uncompress_features_labels(file):
    """
    Uncompress features and labels from a zip file
    :param file: The zip file to extract the data from
    """
    features = []
    labels = []

    with ZipFile(file) as zipf:
        # Progress Bar
        filenames_pbar = tqdm(zipf.namelist(), unit='files')

        # Get features and labels from all files
        for filename in filenames_pbar:
            # Check if the file is a directory
            if not filename.endswith('/'):
                with zipf.open(filename) as image_file:
                    image = Image.open(image_file)
                    image.load()
                    # Load image data as 1 dimensional array
                    # We're using float32 to save on memory space
                    feature = np.array(image, dtype=np.float32).flatten()

                # Get the the letter from the filename.  This is the letter of the image.
                label = os.path.split(filename)[1][0]

                features.append(feature)
                labels.append(label)
    return np.array(features), np.array(labels)

# Get the features and labels from the zip files
train_features, train_labels = uncompress_features_labels('notMNIST_train.zip')
test_features, test_labels = uncompress_features_labels('notMNIST_test.zip')

# Limit the amount of data to work with a docker container
docker_size_limit = 150000
train_features, train_labels = resample(train_features, train_labels, n_samples=docker_size_limit)

# Set flags for feature engineering.  This will prevent you from skipping an important step.
is_features_normal = False
is_labels_encod = False

# Wait until you see that all features and labels have been uncompressed.
print('All features and labels uncompressed.')

# Problem 1 - Implement Min-Max scaling for grayscale image data
def normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    # TODO: Implement Min-Max scaling for grayscale image data
    a, b = 0.1, 0.9
    b_a = b - a
    min_data = 0
    max_data = 255
    data_diff = max_data - min_data
    return a + ((image_data - min_data) * b_a)/(max_data - min_data)

### DON'T MODIFY ANYTHING BELOW ###
# Test Cases
np.testing.assert_array_almost_equal(
    normalize_grayscale(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 255])),
    [0.1, 0.103137254902, 0.106274509804, 0.109411764706, 0.112549019608, 0.11568627451, 0.118823529412, 0.121960784314,
     0.125098039216, 0.128235294118, 0.13137254902, 0.9],
    decimal=3)
np.testing.assert_array_almost_equal(
    normalize_grayscale(np.array([0, 1, 10, 20, 30, 40, 233, 244, 254,255])),
    [0.1, 0.103137254902, 0.13137254902, 0.162745098039, 0.194117647059, 0.225490196078, 0.830980392157, 0.865490196078,
     0.896862745098, 0.9])

if not is_features_normal:
    train_features = normalize_grayscale(train_features)
    test_features = normalize_grayscale(test_features)
    is_features_normal = True

print('Tests Passed!')

if not is_labels_encod:
    # Turn labels into numbers and apply One-Hot Encoding
    encoder = LabelBinarizer()
    encoder.fit(train_labels)
    train_labels = encoder.transform(train_labels)
    test_labels = encoder.transform(test_labels)

    # Change to float32, so it can be multiplied against the features in TensorFlow, which are float32
    train_labels = train_labels.astype(np.float32)
    test_labels = test_labels.astype(np.float32)
    is_labels_encod = True

print('Labels One-Hot Encoded')

assert is_features_normal, 'You skipped the step to normalize the features'
assert is_labels_encod, 'You skipped the step to One-Hot Encode the labels'

# Get randomized datasets for training and validation
train_features, valid_features, train_labels, valid_labels = train_test_split(
    train_features,
    train_labels,
    test_size=0.05,
    random_state=832289)

print('Training features and labels randomized and split.')

# Save the data for easy access
pickle_file = 'notMNIST.pickle'
if not os.path.isfile(pickle_file):
    print('Saving data to pickle file...')
    try:
        with open('notMNIST.pickle', 'wb') as pfile:
            pickle.dump(
                {
                    'train_dataset': train_features,
                    'train_labels': train_labels,
                    'valid_dataset': valid_features,
                    'valid_labels': valid_labels,
                    'test_dataset': test_features,
                    'test_labels': test_labels,
                },
                pfile, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to', pickle_file, ':', e)
        raise

print('Data cached in pickle file.')

All modules imported.
All files downloaded.


100%|██████████| 10001/10001 [00:01<00:00, 6174.27files/s]


All features and labels uncompressed.
Tests Passed!
Labels One-Hot Encoded
Training features and labels randomized and split.
Data cached in pickle file.


In [1]:
%matplotlib inline

# Load the modules
import pickle
import math

import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt

# Reload the data
pickle_file = 'notMNIST.pickle'
with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  train_features = pickle_data['train_dataset']
  train_labels = pickle_data['train_labels']
  valid_features = pickle_data['valid_dataset']
  valid_labels = pickle_data['valid_labels']
  test_features = pickle_data['test_dataset']
  test_labels = pickle_data['test_labels']
  del pickle_data  # Free up memory


print('Data and modules loaded.')

2025-11-03 11:20:13.790195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762165213.817237   70288 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762165213.824945   70288 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762165213.847821   70288 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762165213.847849   70288 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762165213.847851   70288 computation_placer.cc:177] computation placer alr

Data and modules loaded.


In [2]:
# Data has been normalized
total_samples = len(train_features)

# Parameters
learning_rate = 0.003
num_output = 10
epochs = 200
batch_size = 256
steps_per_epoch = int(np.ceil(total_samples / batch_size))

In [3]:
# Cleaning Data
#valid_labels, test_labels = np.array(valid_labels, np.int64), np.array(test_labels, np.int64)
train_data = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
val_features, test_features = np.array(valid_features, np.float32), np.array(test_features, np.float32)

# Data has been normalized

I0000 00:00:1762165223.877899   70288 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6096 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5


In [4]:
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [5]:
features = train_features.shape[1]

In [6]:
#W = tf.Variable(tf.random.normal([features, num_output], name="weights1"))
#B = tf.Variable(tf.zeros([num_output], name="bias1"))

W = {
    "w1": tf.Variable(tf.random.normal([features, 512]), name="weight1"),
    "w2": tf.Variable(tf.random.normal([512, 128]), name="weight2"),
    "w3": tf.Variable(tf.random.normal([128, 32]), name="weight3"),
    "out": tf.Variable(tf.random.normal([32, num_output]), name="weight_out")
}

B = {
    "b1": tf.Variable(tf.zeros([512], name="bias1")),
    "b2": tf.Variable(tf.zeros([128], name="bias2")),
    "b3": tf.Variable(tf.zeros([32], name="bias3")),
    "out": tf.Variable(tf.zeros([num_output], name="bias_out"))
}

optimizer = tf.optimizers.SGD(learning_rate)

# Model
class MyModel(tf.Module):
    def __call__(self, X):
      X = tf.add(tf.matmul(X, W["w1"]), B["b1"])
      X = tf.nn.relu(X)
      X = tf.add(tf.matmul(X, W["w2"]), B["b2"])
      X = tf.nn.relu(X)
      X = tf.add(tf.matmul(X, W["w3"]), B["b3"])
      X = tf.nn.relu(X)
      return tf.nn.softmax(tf.add(tf.matmul(X, W["out"]), B["out"]))

def cross_entropy(y_pred, y_true):
    # It has been one-hot encoded before storing as pickle
    #y_true = tf.one_hot(y_true, depth=num_output)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), 1))

def accuracy(y_pred, y_true):
    if len(y_true.shape) > 1 and y_true.shape[1] > 1:
        y_true = tf.argmax(y_true, axis=1)
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def run_optimizer(X, Y):
    with tf.GradientTape() as g:
        logit = model(X)
        loss = cross_entropy(logit, Y)

    trainable_variables = list(W.values()) + list(B.values())

    gradients = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return None

def batch_data(X, Y, batch_size):
    output_data = []
    sample_size = len(X)
    for step in range(0, sample_size, batch_size):
        start = batch_size * step
        end = batch_size + start
        batch_X = X[start:end]
        batch_Y = Y[start:end]
        yield batch_X, batch_Y

model=MyModel()

checkpoint = tf.train.Checkpoint(model = model)
checkpoint.save("./checkpoints/mymodel")
manager = tf.train.CheckpointManager(checkpoint, "./checkpoints", max_to_keep=3)

In [7]:
# Training
for epoch in range(1, epochs + 1):
    for step, (batch_X, batch_Y) in enumerate(train_data.take(steps_per_epoch), 1):
        run_optimizer(batch_X, batch_Y)

    val_pred = model(valid_features)
    val_loss = cross_entropy(val_pred, valid_labels)
    val_acc = accuracy(val_pred, valid_labels)
    train_pred = model(batch_X)
    train_loss = cross_entropy(train_pred, batch_Y)
    train_acc = accuracy(train_pred, batch_Y)
    manager.save()

    print(f"Epoch: {epoch}, train_loss {train_loss}, train_accuracy {train_acc}, val_loss {val_loss}, val_accuracy {val_acc}")

2025-11-03 11:20:41.345619: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 1, train_loss 15.9472017288208, train_accuracy 0.23046875, val_loss 16.08010482788086, val_accuracy 0.22360000014305115


2025-11-03 11:20:55.063255: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 2, train_loss 14.894847869873047, train_accuracy 0.28125, val_loss 15.113365173339844, val_accuracy 0.27026665210723877
Epoch: 3, train_loss 14.89484977722168, train_accuracy 0.28125, val_loss 14.618520736694336, val_accuracy 0.29413333535194397


2025-11-03 11:21:20.838429: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

Epoch: 32, train_loss 14.490096092224121, train_accuracy 0.30078125, val_loss 14.156024932861328, val_accuracy 0.3166666626930237
Epoch: 33, train_loss 14.004393577575684, train_accuracy 0.32421875, val_loss 13.67917537689209, val_accuracy 0.3397333323955536
Epoch: 34, train_loss 14.085344314575195, train_accuracy 0.3203125, val_loss 13.649806022644043, val_accuracy 0.34119999408721924
Epoch: 35, train_loss 14.813896179199219, train_accuracy 0.28515625, val_loss 13.75751781463623, val_accuracy 0.33613333106040955
Epoch: 36, train_loss 14.409146308898926, train_accuracy 0.3046875, val_loss 13.68655014038086, val_accuracy 0.33933332562446594
Epoch: 37, train_loss 13.680593490600586, train_accuracy 0.33984375, val_loss 13.55478572845459, val_accuracy 0.3457333445549011
Epoch: 38, train_loss 14.00439453125, train_accuracy 0.32421875, val_loss 13.586287498474121, val_accuracy 0.3442666530609131
Epoch: 39, train_loss 12.790142059326172, train_accuracy 0.3828125, val_loss 13.581897735595703, 

2025-11-03 11:33:11.199350: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 64, train_loss 14.247245788574219, train_accuracy 0.3125, val_loss 13.500679969787598, val_accuracy 0.3482666611671448
Epoch: 65, train_loss 12.304438591003418, train_accuracy 0.40625, val_loss 13.582154273986816, val_accuracy 0.3445333242416382
Epoch: 66, train_loss 14.00439453125, train_accuracy 0.32421875, val_loss 13.486298561096191, val_accuracy 0.34906667470932007
Epoch: 67, train_loss 13.275843620300293, train_accuracy 0.359375, val_loss 13.531766891479492, val_accuracy 0.3469333350658417
Epoch: 68, train_loss 13.27584171295166, train_accuracy 0.359375, val_loss 13.591699600219727, val_accuracy 0.3441333472728729
Epoch: 69, train_loss 13.59964370727539, train_accuracy 0.34375, val_loss 13.50054931640625, val_accuracy 0.3485333323478699
Epoch: 70, train_loss 14.32819652557373, train_accuracy 0.30859375, val_loss 13.526128768920898, val_accuracy 0.3472000062465668
Epoch: 71, train_loss 13.761543273925781, train_accuracy 0.3359375, val_loss 13.673370361328125, val_accuracy 0

2025-11-03 11:44:02.116601: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 128, train_loss 8.499776840209961, train_accuracy 0.58984375, val_loss 9.562102317810059, val_accuracy 0.5385333299636841
Epoch: 129, train_loss 10.604484558105469, train_accuracy 0.48828125, val_loss 10.402511596679688, val_accuracy 0.4975999891757965
Epoch: 130, train_loss 9.875931739807129, train_accuracy 0.5234375, val_loss 9.039019584655762, val_accuracy 0.5634666681289673
Epoch: 131, train_loss 9.95688247680664, train_accuracy 0.51953125, val_loss 9.010406494140625, val_accuracy 0.5650666952133179
Epoch: 132, train_loss 10.604484558105469, train_accuracy 0.48828125, val_loss 9.513537406921387, val_accuracy 0.5405333042144775
Epoch: 133, train_loss 9.552130699157715, train_accuracy 0.5390625, val_loss 9.147262573242188, val_accuracy 0.5583999752998352
Epoch: 134, train_loss 9.309280395507812, train_accuracy 0.55078125, val_loss 9.01533317565918, val_accuracy 0.5646666884422302
Epoch: 135, train_loss 10.36163330078125, train_accuracy 0.5, val_loss 9.009541511535645, val_accu

In [8]:
test_pred = model(test_features)
test_loss = cross_entropy(test_pred, test_labels)
test_acc = accuracy(test_pred, test_labels)
print(f"Test_loss {test_loss}, Test_accuracy {test_acc}")

Test_loss 8.052408218383789, Test_accuracy 0.6111999750137329
